<a href="https://colab.research.google.com/github/dkswndms4782/BoostCamp_AI_Tech/blob/main/2_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**2. Word2Vec**
1. 주어진 단어들을 word2vec 모델에 들어갈 수 있는 형태로 만듭니다.
2. CBOW, Skip-gram 모델을 각각 구현합니다.
3. 모델을 실제로 학습해보고 결과를 확인합니다.

### **필요 패키지 import**

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 12.4MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 460kB 55.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
from tqdm import tqdm
from konlpy.tag import Okt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

import torch
import copy
import numpy as np

### **데이터 전처리**



데이터를 확인하고 Word2Vec 형식에 맞게 전처리합니다.  
학습 데이터는 1번 실습과 동일하고, 테스트를 위한 단어를 아래와 같이 가정해봅시다.

In [ ]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."       
]

test_words = ["음식", "맛", "서비스", "위생", "가격"]

Tokenization과 vocab을 만드는 과정은 이전 실습과 유사합니다.

In [ ]:
tokenizer = Okt()

In [ ]:
def make_tokenized(data):
  tokenized = []
  for sent in tqdm(data):
    tokens = tokenizer.morphs(sent, stem=True)
    tokenized.append(tokens)

  return tokenized

In [ ]:
train_tokenized = make_tokenized(train_data)

100%|██████████| 10/10 [00:05<00:00,  1.69it/s]


In [ ]:
word_count = defaultdict(int)

for tokens in tqdm(train_tokenized):
  for token in tokens:
    word_count[token] += 1

100%|██████████| 10/10 [00:00<00:00, 18149.30it/s]


In [ ]:
word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
print(list(word_count))

[('.', 14), ('도', 7), ('이다', 4), ('좋다', 4), ('별로', 3), ('다', 3), ('이', 3), ('너무', 3), ('음식', 3), ('서비스', 3), ('하다', 2), ('방문', 2), ('위생', 2), ('좀', 2), ('더', 2), ('에', 2), ('조금', 2), ('정말', 1), ('맛있다', 1), ('추천', 1), ('기대하다', 1), ('것', 1), ('보단', 1), ('가격', 1), ('비싸다', 1), ('다시', 1), ('가다', 1), ('싶다', 1), ('생각', 1), ('안', 1), ('드네', 1), ('요', 1), ('완전', 1), ('최고', 1), ('!', 1), ('재', 1), ('의사', 1), ('있다', 1), ('만족스럽다', 1), ('상태', 1), ('가', 1), ('개선', 1), ('되다', 1), ('기르다', 1), ('바라다', 1), ('맛', 1), ('직원', 1), ('분들', 1), ('친절하다', 1), ('기념일', 1), ('분위기', 1), ('전반', 1), ('적', 1), ('으로', 1), ('짜다', 1), ('저', 1), ('는', 1), ('신경', 1), ('써다', 1), ('불쾌하다', 1)]


In [ ]:
w2i = {}
for pair in tqdm(word_count):
  if pair[0] not in w2i:
    w2i[pair[0]] = len(w2i)

100%|██████████| 60/60 [00:00<00:00, 64693.63it/s]


In [ ]:
print(train_tokenized)
print(w2i)

[['정말', '맛있다', '.', '추천', '하다', '.'], ['기대하다', '것', '보단', '별로', '이다', '.'], ['다', '좋다', '가격', '이', '너무', '비싸다', '다시', '가다', '싶다', '생각', '이', '안', '드네', '요', '.'], ['완전', '최고', '이다', '!', '재', '방문', '의사', '있다', '.'], ['음식', '도', '서비스', '도', '다', '만족스럽다', '.'], ['위생', '상태', '가', '좀', '별로', '이다', '.', '좀', '더', '개선', '되다', '기르다', '바라다', '.'], ['맛', '도', '좋다', '직원', '분들', '서비스', '도', '너무', '친절하다', '.'], ['기념일', '에', '방문', '하다', '음식', '도', '분위기', '도', '서비스', '도', '다', '좋다', '.'], ['전반', '적', '으로', '음식', '이', '너무', '짜다', '.', '저', '는', '별로', '이다', '.'], ['위생', '에', '조금', '더', '신경', '써다', '좋다', '.', '조금', '불쾌하다', '.']]
{'.': 0, '도': 1, '이다': 2, '좋다': 3, '별로': 4, '다': 5, '이': 6, '너무': 7, '음식': 8, '서비스': 9, '하다': 10, '방문': 11, '위생': 12, '좀': 13, '더': 14, '에': 15, '조금': 16, '정말': 17, '맛있다': 18, '추천': 19, '기대하다': 20, '것': 21, '보단': 22, '가격': 23, '비싸다': 24, '다시': 25, '가다': 26, '싶다': 27, '생각': 28, '안': 29, '드네': 30, '요': 31, '완전': 32, '최고': 33, '!': 34, '재': 35, '의사': 36, '있다': 37, '만족스럽다': 38, '상태

실제 모델에 들어가기 위한 input을 만들기 위해 `Dataset` 클래스를 정의합니다.

In [ ]:
class CBOWDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.y.append(id)

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수, 2 * window_size)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [ ]:
class SkipGramDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.y += (token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.x += [id] * 2 * window_size

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

각 모델에 맞는 `Dataset` 객체를 생성합니다.

In [ ]:
cbow_set = CBOWDataset(train_tokenized)
skipgram_set = SkipGramDataset(train_tokenized)
print(list(skipgram_set))

100%|██████████| 10/10 [00:00<00:00, 12561.56it/s]

[(tensor(0), tensor(17)), (tensor(0), tensor(18)), (tensor(0), tensor(19)), (tensor(0), tensor(10)), (tensor(19), tensor(18)), (tensor(19), tensor(0)), (tensor(19), tensor(10)), (tensor(19), tensor(0)), (tensor(22), tensor(20)), (tensor(22), tensor(21)), (tensor(22), tensor(4)), (tensor(22), tensor(2)), (tensor(4), tensor(21)), (tensor(4), tensor(22)), (tensor(4), tensor(2)), (tensor(4), tensor(0)), (tensor(23), tensor(5)), (tensor(23), tensor(3)), (tensor(23), tensor(6)), (tensor(23), tensor(7)), (tensor(6), tensor(3)), (tensor(6), tensor(23)), (tensor(6), tensor(7)), (tensor(6), tensor(24)), (tensor(7), tensor(23)), (tensor(7), tensor(6)), (tensor(7), tensor(24)), (tensor(7), tensor(25)), (tensor(24), tensor(6)), (tensor(24), tensor(7)), (tensor(24), tensor(25)), (tensor(24), tensor(26)), (tensor(25), tensor(7)), (tensor(25), tensor(24)), (tensor(25), tensor(26)), (tensor(25), tensor(27)), (tensor(26), tensor(24)), (tensor(26), tensor(25)), (tensor(26), tensor(27)), (tensor(26), tens

### **모델 Class 구현**

차례대로 두 가지 Word2Vec 모델을 구현합니다.  


*   `self.embedding`: `vocab_size` 크기의 one-hot vector를 특정 크기의 `dim` 차원으로 embedding 시키는 layer.
*   `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.


In [ ]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, dim):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x):  # x: (B, 2W)
    embeddings = self.embedding(x)  # (B, 2W, d_w)
    embeddings = torch.sum(embeddings, dim=1)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

In [ ]:
class SkipGram(nn.Module):
  def __init__(self, vocab_size, dim):
    super(SkipGram, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x): # x: (B)
    embeddings = self.embedding(x)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

두 가지 모델을 생성합니다.

In [ ]:
cbow = CBOW(vocab_size=len(w2i), dim=256)
skipgram = SkipGram(vocab_size=len(w2i), dim=256)

### **모델 학습**

다음과 같이 hyperparamter를 세팅하고 `DataLoader` 객체를 만듭니다.

In [ ]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

cbow_loader = DataLoader(cbow_set, batch_size=batch_size)
skipgram_loader = DataLoader(skipgram_set, batch_size=batch_size)

첫번째로 CBOW 모델 학습입니다.

In [ ]:
cbow.train()
cbow = cbow.to(device)
optim = torch.optim.SGD(cbow.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(cbow_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = cbow(x)  # (B, V)
 
    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  6%|▋         | 1/16 [00:00<00:02,  5.80it/s]

##################################################
Epoch: 1
Train loss: 4.694368839263916
Train loss: 4.9900994300842285
Train loss: 4.380759239196777
Train loss: 4.762627601623535
Train loss: 4.068875312805176
Train loss: 4.871387004852295
Train loss: 3.8936617374420166
Train loss: 6.299971580505371
Train loss: 4.343380928039551
Train loss: 4.802947998046875
Train loss: 5.04029655456543
Train loss: 4.32087516784668
Train loss: 4.892443656921387
Train loss: 4.772637844085693
Train loss: 4.9122467041015625
Train loss: 5.341496467590332


100%|██████████| 16/16 [00:00<00:00, 661.20it/s]

##################################################
Epoch: 2
Train loss: 4.520066738128662
Train loss: 4.85860538482666
Train loss: 4.2701802253723145
Train loss: 4.63629150390625
Train loss: 3.930943727493286
Train loss: 4.54085111618042
Train loss: 3.7270731925964355
Train loss: 6.165835380554199
Train loss: 4.22045373916626
Train loss: 4.598281383514404
Train loss: 4.827646255493164
Train loss: 3.8973984718322754
Train loss: 4.748327255249023
Train loss: 4.6352434158325195
Train loss: 4.749530792236328
Train loss: 5.203269958496094
##################################################
Epoch: 3
Train loss: 4.349672317504883
Train loss: 4.728671550750732
Train loss: 4.160826683044434
Train loss: 4.51141357421875
Train loss: 3.7954578399658203
Train loss: 4.219063758850098
Train loss: 3.5648701190948486
Train loss: 6.033149719238281
Train loss: 4.103801250457764
Train loss: 4.398245811462402
Train loss: 4.624841690063477
Train loss: 3.4940457344055176
Train loss: 4.606417179107666
Train lo

다음으로 Skip-gram 모델 학습입니다.

In [ ]:
skipgram.train()
skipgram = skipgram.to(device)
optim = torch.optim.SGD(skipgram.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  for batch in tqdm(skipgram_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = skipgram(x)  # (B, V)

    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

  0%|          | 0/64 [00:00<?, ?it/s]

##################################################
Epoch: 1
Train loss: 4.586830139160156
Train loss: 4.703792572021484
Train loss: 4.144767761230469
Train loss: 4.104708671569824
Train loss: 4.623551845550537
Train loss: 3.9838764667510986
Train loss: 4.206255912780762
Train loss: 4.372400283813477
Train loss: 3.7087604999542236
Train loss: 4.591941833496094
Train loss: 4.648377418518066
Train loss: 4.642399787902832
Train loss: 4.081422805786133
Train loss: 4.6173248291015625
Train loss: 4.162888526916504
Train loss: 4.193804740905762
Train loss: 4.499189376831055
Train loss: 4.0991387367248535
Train loss: 4.107501983642578
Train loss: 3.9965221881866455
Train loss: 4.075960159301758
Train loss: 3.816110610961914
Train loss: 4.5798821449279785
Train loss: 4.245028972625732
Train loss: 4.414789199829102
Train loss: 4.211215496063232
Train loss: 4.134530067443848
Train loss: 4.290428638458252
Train loss: 4.138589382171631
Train loss: 4.534265995025635
Train loss: 4.374664306640625
Trai

  0%|          | 0/64 [00:00<?, ?it/s]

Train loss: 4.140476226806641
Train loss: 4.311333656311035
Train loss: 3.658367156982422
Train loss: 4.530157089233398
Train loss: 4.56845760345459
Train loss: 4.5989274978637695
Train loss: 4.038201332092285
Train loss: 4.552997589111328
Train loss: 4.089859962463379
Train loss: 4.124125957489014
Train loss: 4.438695430755615
Train loss: 4.042344093322754
Train loss: 4.047447204589844
Train loss: 3.94342041015625
Train loss: 3.8815951347351074
Train loss: 3.6574161052703857
Train loss: 4.462964057922363
Train loss: 4.187769412994385
Train loss: 4.330717086791992
Train loss: 4.085813045501709
Train loss: 4.04412841796875
Train loss: 4.2421088218688965
Train loss: 4.0618062019348145
Train loss: 4.462359428405762
Train loss: 4.312851905822754
Train loss: 4.430353164672852
Train loss: 4.017167091369629
Train loss: 4.094740867614746
Train loss: 4.026514530181885
Train loss: 4.533900737762451
Train loss: 4.149663925170898
Train loss: 3.8663320541381836
Train loss: 3.9822964668273926
Train 

100%|██████████| 64/64 [00:00<00:00, 705.43it/s]

Train loss: 3.9640092849731445
Train loss: 3.955781936645508
Train loss: 4.1952314376831055
Train loss: 3.986517906188965
Train loss: 4.391817092895508
Train loss: 4.251619338989258
Train loss: 4.367560863494873
Train loss: 3.970226287841797
Train loss: 4.025598526000977
Train loss: 3.9761080741882324
Train loss: 4.4661478996276855
Train loss: 4.032526016235352
Train loss: 3.7954776287078857
Train loss: 3.912243604660034
Train loss: 4.413867950439453
Train loss: 4.327066421508789
Train loss: 3.974008560180664
Train loss: 3.6560230255126953
Train loss: 3.680647850036621
Train loss: 3.3327994346618652
Train loss: 3.6570396423339844
Train loss: 3.72416353225708
Train loss: 4.035982608795166
Train loss: 4.033868789672852
Train loss: 4.187726020812988
Train loss: 3.8151636123657227
Train loss: 3.7803144454956055
Train loss: 3.9384665489196777
Train loss: 3.703827381134033
Train loss: 4.240733623504639
Train loss: 4.3831682205200195
Train loss: 4.042823314666748
Train loss: 4.433591842651367

### **테스트**

학습된 각 모델을 이용하여 test 단어들의 word embedding을 확인합니다.

In [ ]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = cbow.embedding(input_id)

  print(f"Word: {word}")
  print(emb.squeeze(0))

Word: 음식
tensor([ 0.1205, -0.6906,  1.2125,  0.8747,  1.1603,  1.6634, -0.2847, -0.5266,
         1.0243, -1.4299, -1.8155,  0.0230, -0.5703, -0.8459, -0.8317,  1.7981,
         0.6013,  0.6252,  0.1489,  1.0409, -0.2567, -0.2421, -0.3163,  0.0777,
         0.0063, -0.7628,  0.3231,  1.0551,  1.1203,  0.3657, -0.7725,  0.1117,
         0.6542,  1.2464, -0.3078, -0.6025,  0.8577,  0.0366,  0.2887,  0.3252,
        -0.3527, -0.8851, -0.3600, -0.1836,  0.1824,  0.9931, -0.3883,  1.3692,
        -1.3769, -0.1664,  0.1763,  0.2280,  2.6283, -0.2433, -1.0066, -2.0993,
         0.5470,  0.3769,  0.9690, -0.1904, -0.2749, -2.1109,  0.9469,  0.9810,
        -0.5039, -0.0865, -0.9831, -0.2973, -1.3241,  0.1638,  1.2461, -1.0169,
        -0.1694, -0.3969, -0.8298, -0.7485, -1.2220, -0.5490, -0.3973, -1.3511,
        -0.6827, -1.4021,  0.1970,  0.0616, -0.5831, -0.3808, -0.4074, -0.1162,
        -1.6924,  0.8244,  0.6510, -1.7786, -1.1903,  0.7170,  0.3414, -0.1464,
         1.0889, -0.9081, -0.44

In [ ]:
for word in test_words:
  input_id = torch.LongTensor([w2i[word]]).to(device)
  emb = skipgram.embedding(input_id)

  print(f"Word: {word}")
  print(max(emb.squeeze(0)))

NameError: ignored

In [ ]:
!apt-get install -qq texlive texlive-xetex texlive-latex-extra pandoc
!pip install -qq pypandoc

from google.colab import drive
drive.mount('/content/drive')

!jupyter nbconvert --to PDF '/content/drive/My Drive/Colab Notebooks/1_naive_bayes.ipynb의 사본'